# Playground

In [ ]:
import pandas as pd

# Existing DataFrame
existing_data = {
    'dict1': {'a': 1, 'b': 2, 'c': 3},
    'dict2': {'d': 4, 'e': 5, 'f': 6},
    'dict3': {'g': 7, 'h': 8, 'i': 9}
}
df = pd.DataFrame(existing_data)

# New data dictionary with different keys and integer index
new_data = {
    'dict4': {"j": 10, 'k': 11, 'l': 12},
    'dict5': {0: 13, 1: 14, 2: 15}
}

# Create a new DataFrame from the new data dictionary
new_df = pd.DataFrame(new_data)

# Concatenate the existing DataFrame with the new DataFrame
df = pd.concat([df, new_df], axis=1)

# Reset the index of the resulting DataFrame to integers
df = df.T
df = df.reset_index(drop=True)

df
